In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [10]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [11]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [5]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Subject: '
                                                                          'Re: '
                                                                          'Meeting '
                                                                          'reschedule\n'
                                                                          '\n'
                                                                          'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'letting '

In [6]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Subject: Re: Meeting reschedule\n\nHi John,\n\nNo problem—thanks for letting me know. I’m happy to reschedule. Would any of these times tomorrow work for you?\n\n- 2:00 PM\n- 4:00 PM\n- 5:30 PM\n\nIf none of these fit, please suggest a couple of alternatives and I’ll adjust.\n\nBest regards,\nSeán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Subject: Re: Meeting reschedule\\n\\nHi John,\\n\\nNo problem—thanks for letting me know. I’m happy to reschedule. Would any of these times tomorrow work for you?\\n\\n- 2:00 PM\\n- 4:00 PM\\n- 5:30 PM\\n\\nIf none of these fit, please suggest a couple of alternatives and I’ll adjust.\\n\\nBest regards,\\nSeán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='74ac59d08e02b378fa9d8a71bab787ed')]


In [7]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Subject: Re: Meeting reschedule

Hi John,

No problem—thanks for letting me know. I’m happy to reschedule. Would any of these times tomorrow work for you?

- 2:00 PM
- 4:00 PM
- 5:30 PM

If none of these fit, please suggest a couple of alternatives and I’ll adjust.

Best regards,
Seán


## Approve

In [8]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='cf21f51f-c954-4bc8-bba6-a1bc49a6232c'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1427, 'prompt_tokens': 157, 'total_tokens': 1584, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1408, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-ClgI5Zr1k468AFDse1PYQAIS42mCy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0ecf-2d2e-7321-9f3c-d24c65b10fd7-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call

## Reject

In [12]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Re: '
                                                                          'Meeting '
                                                                          'tomorrow\n'
                                                                          '\n'
                                                                          'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thanks '
                                                                          'for '
                                                                          'the '
                                                                          'heads '
                                                                          'up. '
               

In [13]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Re: Meeting tomorrow

Hi John,

Thanks for the heads up. No problem—let's reschedule. Are either of these times good for you?

- Tomorrow at 3:00 PM
- Tomorrow at 4:00 PM

If neither works, please share two or three alternative times and I’ll do my best to accommodate.

Your merciful leader, Seán


## Edit

In [14]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='9d914c5e-e6cb-4577-923c-c8dcaa0479ca'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 211, 'prompt_tokens': 157, 'total_tokens': 368, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-ClgNMkTpjyydikaFR8eliFRxAqitn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b0ed4-2c31-7922-8ffb-cb12f2b95e31-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_h7